## Проект "Анализ веб-документов" Техносфера Весна 2021

In [50]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

### Посмотрим на наши трейновые данные

In [51]:
traindf = pd.read_csv('data/train_groups.csv', sep=',', index_col='pair_id')
print(traindf.shape)
traindf.head(5)

(11690, 3)


,group_id,doc_id,target
pair_id,,,
1,1,15731,0
2,1,14829,0
3,1,15764,0
4,1,17669,0
5,1,14852,0


In [52]:
traindf.shape

(11690, 3)

# Идея 0 

Посмотрим на каждые заголовки, построим множество признаков, основанных на количестве схожих слов в заголовках.
Будем брать топ 15 в списке

## Решение 0
Повторим решение со второй домашней работы

1. Прочитаем все заголовки

In [53]:
doc_to_title = {}

with open('data/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_to_title[int(data[0])] = '' if len(data) == 1 else data[1]

filesdf = pd.DataFrame.from_dict(doc_to_title, orient='index', columns=['title'])
filesdf.head(4)

,title
15731,ВАЗ 21213 | Замена подшипников ступицы | Нива
14829,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п..."
15764,Купить ступица Лада калина2. Трансмиссия - пер...
17669,Классика 21010 - 21074


2. Прочитаем трейн и добавим заголовки к нему

In [54]:
def add_titles(df):
    df['title'] = filesdf.loc[df['doc_id'].values].values

traindf = pd.read_csv('data/train_groups.csv', index_col='pair_id')
add_titles(traindf)
traindf.head(4)

,group_id,doc_id,target,title
pair_id,,,,
1,1,15731,0,ВАЗ 21213 | Замена подшипников ступицы | Нива
2,1,14829,0,"Ваз 2107 оптом в Сочи. Сравнить цены, купить п..."
3,1,15764,0,Купить ступица Лада калина2. Трансмиссия - пер...
4,1,17669,0,Классика 21010 - 21074


3. Найдем признаки. Посчитаем общие слова для всех групп и всех веб-страниц в заголовке. Как признаки для веб-страницы, возьмем значения топ-15  пересечений с другими страницами из группы. 

In [55]:
X_train, y_train, groups = [], [], []

In [56]:
def collect_titles_for_train(docs):
    titles = docs.title
    y_train.extend(docs.target.to_list())
    groups.extend(docs.group_id.to_list())
    for i, title in enumerate(titles):
        words = set(title.strip().split())
        distances = []
        for j, another_title in enumerate(titles):
            if i == j:
                continue
            another_words = set(another_title.strip().split())
            distances.append(len(words.intersection(another_words)))
        fueatures_to_add = (-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])
        X_train.append(-np.sort(-fueatures_to_add))

In [57]:
traindf.groupby('group_id').apply(collect_titles_for_train);
X, y, groups = np.asarray(X_train), np.asarray(y_train), np.asarray(groups)

4. Отмасштабируем входные данные

In [58]:
from sklearn.preprocessing import StandardScaler

In [59]:
scaler = StandardScaler().fit(X, y)
X = scaler.transform(X);

5. Подберем гиперпараметры

In [60]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [61]:
opt_params = {
    'n_jobs': -1
}

In [62]:
def ValScore(clf, X_val, y_val, groups, n_splits=5, *args, **kwargs):
    clf = clf(*args, **kwargs)
    kf = GroupKFold(n_splits=n_splits)

    scores = []
    for train, test in kf.split(X_val, y_val, groups=groups):
        clf.fit(X_val[train], y_val[train])
        scores.append(f1_score(y_pred=clf.predict(X_val[test]),
                               y_true=y_val[test]))
    return np.asarray(scores)

In [63]:
def FindParams(clf, X_val, y_val, groups, param_name, param_range, known_params=opt_params):
    mean_scores = []

    for param in param_range:
        kwargs = known_params
        kwargs.update({param_name: param})
        
        scores = ValScore(clf, X_val, y_val, groups, **kwargs)
        mean_scores.append(scores.mean())

    opt_param = param_range[np.argmax(mean_scores)]
    return opt_param

In [64]:
grid = {
    'max_iter': [500, 1000, 2000, 3000, 5000],
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'alpha': np.logspace(4, -4, 10)
}

for param in grid:
    param_range = grid[param]
    opt_model_type = FindParams(SGDClassifier, X, y, groups, param, param_range, opt_params)
    opt_params.update({param: opt_model_type})

In [65]:
opt_params

{'n_jobs': -1, 'max_iter': 3000, 'loss': 'hinge', 'alpha': 0.0001}

### Считываем test и делаем predict

In [66]:
testdf = pd.read_csv('data/test_groups.csv', index_col='pair_id')
add_titles(testdf)
testdf.head(4)

,group_id,doc_id,title
pair_id,,,
11691,130,6710,КАК ПРОПИСАТЬ АДМИНКУ В КС 1.6 СЕБЕ ИЛИ ДРУГУ ...
11692,130,4030,Скачать: SGL-RP доработка | Слив мода [MySQL] ...
11693,130,5561,Как прописать админку в кс 1.6 - Counter-Strik...
11694,130,4055,Как прописать простую админку в кс 1 6


In [67]:
X_test = []

In [68]:
def collect_titles_for_test(docs):
    titles = docs.title
    for i, title in enumerate(titles):
        words = set(title.strip().split())
        distances = []
        for j, another_title in enumerate(titles):
            if i == j:
                continue
            another_words = set(another_title.strip().split())
            distances.append(len(words.intersection(another_words)))
        fueatures_to_add = (-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])
        X_test.append(-np.sort(-fueatures_to_add))

In [69]:
testdf.groupby('group_id').apply(collect_titles_for_test);
X_test = np.asarray(X_test)

In [70]:
X_test = scaler.transform(X_test)

In [71]:
clf = SGDClassifier(**opt_params).fit(X, y)
testdf['target'] = clf.predict(X_test)

In [72]:
testdf.drop(columns=['group_id', 'doc_id', 'title']).to_csv('solution0.csv')

### Итоговый скор 0.51976
Надо что-то получше

# Идея 1

Посмотрим на слова в получившихся данных.

In [73]:
from collections import Counter

In [74]:
wordCounter = Counter()
for title in filesdf['title'].values:
    wordCounter.update(title.strip().split())
wordCounter.most_common(5)

[('-', 11915), ('в', 6517), ('и', 5654), ('|', 4754), ('на', 4269)]

Явно среди заголовков оказалось куча мусора и бесполезной информации.
Давайте предобработаем заголовки.

In [75]:
from nltk import corpus, word_tokenize
from string import digits, punctuation
from pymorphy2 import MorphAnalyzer
import re

In [76]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

chars_to_remove = digits + punctuation + "–—«»№•❤★✿╬·"
morph = MorphAnalyzer()
RU_stopwords = set(corpus.stopwords.words('russian'))
EN_stopwords = set(corpus.stopwords.words('english'))
GE_stopwords = set(corpus.stopwords.words('german'))
stopwords = RU_stopwords.union(EN_stopwords.union(GE_stopwords))

In [77]:
def process_text(text):
    if text is None:
        return
    text = emoji_pattern.sub(r'', text)
    text = text.translate(str.maketrans('', '', chars_to_remove))
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords and len(word) > 1]
    words = [morph.parse(word)[0].normal_form for word in words]
    return words

In [78]:
filesdf['title_words'] = filesdf['title'].apply(process_text)

KeyboardInterrupt: 

In [ ]:
filesdf.head(5)

Проверим как изменились наши заголовки

In [ ]:
new_wordCounter = Counter()
for words in filesdf['title_words'].values:
    new_wordCounter.update(words)
new_wordCounter.most_common(10)

Уже лучше, попробуем запустить решение 0 на этих данных

## Решение 1

In [ ]:
def add_title_words(df):
    df['title_words'] = [words for words in filesdf.loc[df['doc_id'].values]['title_words']]

In [ ]:
add_title_words(traindf)
traindf.head(4)

In [ ]:
X_train, y_train, groups = [], [], []

In [ ]:
def collect_words_impl__(train_matrix, words):
    for i, current_words in enumerate(words):
        distances = []
        for j, another_words in enumerate(words):
            if i == j:
                continue
            distances.append(len(set(current_words) & set(another_words)))
        fueatures_to_add = (-np.partition(-np.asarray(distances, dtype='int'), 15)[:15])
        train_matrix.append(-np.sort(-fueatures_to_add))

In [ ]:
def collect_title_words_for_train(docs):
    y_train.extend(docs.target.to_list())
    groups.extend(docs.group_id.to_list())
    collect_words_impl__(X_train, docs.title_words)

In [ ]:
traindf.groupby('group_id').apply(collect_title_words_for_train);
X, y, groups = np.asarray(X_train), np.asarray(y_train), np.asarray(groups)

In [ ]:
scaler = StandardScaler().fit(X, y)
X = scaler.transform(X);

In [ ]:
opt_params = {
    'n_jobs': -1
}

In [ ]:
for param in grid:
    param_range = grid[param]
    opt_model_type = FindParams(SGDClassifier, X, y, groups, param, param_range, opt_params)
    opt_params.update({param: opt_model_type})

In [ ]:
opt_params

### Считываем test и делаем predict

In [ ]:
X_test = []

In [ ]:
def collect_title_words_for_test(docs):
    collect_words_impl__(X_test, docs.title_words)

In [ ]:
testdf = pd.read_csv('data/test_groups.csv', index_col='pair_id')
add_title_words(testdf)
testdf.head(4)

In [ ]:
testdf.groupby('group_id').apply(collect_title_words_for_test);
X_test = np.asarray(X_test)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
clf = SGDClassifier(**opt_params).fit(X, y)
testdf['target'] = clf.predict(X_test)

In [ ]:
testdf.drop(columns=['group_id', 'doc_id', 'title_words']).to_csv('solution1.csv')

### Итоговый скор 0.63423
Ура, побили бейзлайн!

# Идея 2

Явно самих заголовков не хватает. Давайте обкачаем наши данные.

In [ ]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import codecs

Уберем ненужную колонку с заголовком

In [ ]:
traindf.drop(columns=['title'], inplace=True)
traindf.head()

Добавим имя URL без домена и путь.

In [ ]:
def process_url(url):
    first_slash = url.find('/')
    if first_slash != -1:
        last_dot = url[:first_slash].rfind('.')
        rest_url = url[first_slash + 1:]
    else:
        last_dot = url.rfind('.')
        rest_url = ""
    name = url[:last_dot]
    return {
        "url_name": name,
        "url_path": process_text(rest_url)
    }

Из головы достанем верификацию от гугла, а также информацию о ключевых словах и описании

Интуиция подсказывает, что в keywords и desription будут лежать примерно одни и те же слова. Вместо двух листов использовать один словарь.

In [ ]:
def process_head(head):
    verified = int(head.find('meta', attrs={'name': "google-site-verification"}) is not None)

    keywords = head.find('meta', attrs={'name': "keywords"})
    keywords = [] if keywords is None else process_text(keywords['content'])

    description = head.find('meta', attrs={'name': "description"})
    description = [] if description is None else process_text(description['content'])
    
    return {
        'verified': verified,
        'auxiliary': set(keywords).union(description)
    }

Из тела возьмем весь читабельный текст

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def get_visible_text(body):
    texts = body.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

def process_body(body):
    return {
        "text": process_text(get_visible_text(body))
    }

Функция обработки страницы

In [ ]:
def process_page(file):
    soup = BeautifulSoup(file, 'html.parser')
    page_info = dict()
    page_info.update(process_head(soup.find('head')))
    page_info.update(process_body(soup.find('body')))
    return page_info

Посмотрим на какой-нибудь файл

In [ ]:
with codecs.open('content/1.dat', 'r', 'utf-8') as f:
     print(process_page(f))

Слов получилось много, держать для каждого файла эти слова в памяти будет очень затратно. Будем сразу создавать признаки для наших файлов и держать их в таблице.

# Идея 3

Также очевидно, что использовать метрику наибольшего количества похожих слов из первых двух решений будет странновато. Есть такие слова как "как", "ваш" и др., которые встречаются в документах чаще остальных и не являются стоп-словами. Идея состоим в том, чтобы в качестве признаков использовать CountVectorizer на title_words, auiiliary, так как чаще всего эти данные поменьше и логичнее смотреть на количество в их схожести, и TfIdfVectorizer для всего остального текста на сайте.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Попробуем получить признаки для первой группы из трейна:

In [ ]:
groupdf = traindf[traindf.group_id == 1].set_index("doc_id").drop(columns=["group_id"])
groupdf.head()

In [ ]:
for doc_id in tqdm(groupdf.index):
    with codecs.open(f'content/{doc_id}.dat', 'r', 'utf-8') as f:
        data = process_page(f)
        groupdf.loc[doc_id, data.keys()] = data.values()

In [ ]:
groupdf.head(4)

In [ ]:
gcopy = groupdf.copy()

Для больших матриц векторайзеры возвращают sparce матрицы, объединим их

In [ ]:
from scipy import sparse

In [ ]:
cvect = CountVectorizer(analyzer=lambda x: x, max_features=20)
tfidfvect = TfidfVectorizer(analyzer=lambda x: x, max_features=80)

In [ ]:
title_matr = cvect.fit_transform(gcopy.pop('title_words').values)
aux_matr   = cvect.fit_transform(gcopy.pop('auxiliary').values)
text_matr  = tfidfvect.fit_transform(gcopy.pop('text').values)
ver_col = gcopy.pop('verified').values[:, np.newaxis].astype(int)

In [ ]:
train = sparse.hstack((ver_col, title_matr, aux_matr, text_matr))
train.shape

Проверим на этой группе хорошо ли получится предсказывать

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, y = train, groupdf['target'].values
X_train, X_val, y_train, y_val = train_test_split(train, y, random_state=10)
groups = np.ones(X_train.shape[0]).astype(int)

In [ ]:
scaler = StandardScaler(with_mean=False).fit(X_train, y_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
opt_params = {
    'n_jobs': -1
}

In [ ]:
grid = {
    'metric': ['euclidean', 'cosine'],
    'k_neighbours': np.arange(1, 30)
}

for param in grid:
    param_range = grid[param]
    opt_model_type = FindParams(param, param_range, KNeighborsClassifier, opt_params)
    opt_params.update({param: opt_model_type})

In [ ]:
clf = KNeighborsClassifier(metric='cosine').fit(X_train, y_train)
y_pred = clf.predict(X_val)
f1_score(y_pred, y_val)

Обкачивать будем многопоточно

In [ ]:
from multiprocessing.dummy import Pool, Queue

Держать в памяти обкаченные данные каждого документа довольно бредово, так как не хватит памяти. Давайте сразу будем заполнять признаки для трейна. Заполним очередь трейновыми группами.

In [ ]:
queue = Queue()
for group_id in traindf.group_id.unique():
    queue.put(group_id)

Мы знаем, что в двух разных группах может быть один и тот же документ. Это может нам выстрелить в ногу в при работе с многопоточными программами. Выход: можно открывать файл под локом, сохранять его данные в суп, выходить из лока и обрабатывать признаки.

In [ ]:
# def process_page_wrapper(i):
#     while not queue.empty():
#         group_id = queue.get()

#         groupdf = traindf[traindf.group_id == group_id]
#         with lock:
#             with codecsecs.open(f'content/{doc_id}.dat', 'r', 'utf-8') as f:
#                 current_data = process_page(f)

#             with lock:
#                 filesdf.loc[doc_id, current_data.keys()] = current_data.values()
#                 pbar.update(1)


# with Pool(processes=6) as pool, tqdm(total=queue.qsize()) as pbar:
#     lock = pbar.get_lock()
#     pool.map(get_features_wrapper, range(pool._processes))

In [ ]:
filesdf.loc[1, current_data.keys()] = current_data.values()a

In [ ]:
filesdf.loc[1]

In [ ]:
Идея состоит в том, чтобы собрать

## Решение 2

In [ ]:
with codecs.open(f'content/1.dat', 'r', 'utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [ ]:
traindf['doc_id']